In [1]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [2]:
import geopandas as gp
from shapely.geometry import Polygon

In [11]:
%matplotlib inline

In [117]:
polys1 = [
    Polygon([(0,0), (0,2), (2,2), (2,0)]),
    Polygon([(0,0), (0,-2), (-2,-2), (-2,0)]),
    
]
polys1=gp.GeoDataFrame(polys1, columns=['geometry'])

In [118]:
polys2 = [
    Polygon([(0,0), (0,1), (1,1), (1,0)]),
    Polygon([(1,0), (2,0), (2,1), (1,1)]),
    Polygon([(1,1), (1,2), (2,2), (2,1)]),
    Polygon([(0,0), (0,-1), (-1,-1), (-1,0)]),
    
]
polys2= gp.GeoDataFrame(polys2, columns=['geometry'])

In [141]:

def polys2polys(polys1, polys2, pname1='poly1', pname2='poly2', cur_epsg=None, area_epsg=None, intersection_only=True):
    do_crs_transform = True

    # make sure CRS is set correctly
    if cur_epsg is None and polys1.crs is None and polys2.crs is None: 
        if area_epsg is None:
            do_crs_transform = False
            print("No current epsg is specified. Area is computed directed in the current coordinates")
        else:
            raise ValueError('area epsg is specified, but the polygons have no CRS')
    
    if do_crs_transform:
        if area_epsg is None:
            raise ValueError(
                'Need to do area transform, but area is not specified. '
                f"cur_epsg is {cur_epsg}, polys1.crs is {polys1.crs}, polys2.crs is {polys2.crs}"
            )
        if polys1.crs is None: polys1.crs = {'init': 'epsg:%d'%cur_epsg, 'no_defs': True}
        if polys2.crs is None: polys2.crs = {'init': 'epsg:%d'%cur_epsg, 'no_defs': True}
    
    # get intersections between polys1 and polys2
    ps1tops2 = gp.sjoin(polys1, polys2)
    itxns = []
    for li, row in ps1tops2.iterrows():
        itxn = polys2.loc[row.index_right].geometry.intersection(polys1.loc[li].geometry)
        itxns.append({pname1: li, pname2: row.index_right, 'geometry': itxn})
    itxns = gp.GeoDataFrame(itxns)
    
    # get area of the intersections
    if do_crs_transform:
        itxns.crs = polys1.crs
        itxns_for_area = itxns.to_crs(epsg=area_epsg)
    else:
        itxns_for_area = itxns  
    itxns['iarea'] = itxns_for_area.geometry.apply(lambda x: x.area)
    itxns.drop(itxns[itxns['iarea']==0].index, inplace=True)
    
    # compute the weight
    if intersection_only:
        polys1_area = itxns.groupby(pname1).apply(lambda x: x['iarea'].sum()).to_frame()
    else:
        polys1_area = polys1.to_crs(epsg=area_epsg).geometry.apply(lambda x: x.area).to_frame()
        polys1_area.index.name = pname1
    polys1_area = polys1_area
    polys1_area.columns=[pname1+'_area']
    polys1_area.reset_index(inplace=True)
    itxns = itxns.merge(polys1_area)
    itxns['weight'] = itxns['iarea']/itxns[pname1+'_area']
    return gp.pd.DataFrame(itxns[[pname1, pname2, 'weight']])
          
i = polys2polys(polys1, polys2, cur_epsg=4326, area_epsg=6362, intersection_only=False)
i

cur_epsg is 4326, polys1.crs is {'init': 'epsg:4326', 'no_defs': True}, polys2.crs is {'init': 'epsg:4326', 'no_defs': True}


,geometry,poly1,poly2,iarea,poly1_area,weight
0,"POLYGON ((0 0, 0 -1, -1 -1, -1 0, 0 0))",1,3,1.443216e+10,5.813894e+10,0.248236
1,"POLYGON ((0 0, 0 1, 1 1, 1 0, 0 0))",0,0,1.423361e+10,5.655007e+10,0.251699
2,"POLYGON ((2 0, 1 0, 1 1, 2 1, 2 0))",0,1,1.423361e+10,5.655007e+10,0.251699
3,"POLYGON ((1 1, 1 2, 2 2, 2 1, 1 1))",0,2,1.404211e+10,5.655007e+10,0.248313


In [132]:
i

,geometry,poly1,poly2,area,poly1_area,weight
0,"POLYGON ((0 0, 0 -1, -1 -1, -1 0, 0 0))",1,3,1.239266e+10,4.957819e+10,0.249962
1,"POLYGON ((0 0, 0 1, 1 1, 1 0, 0 0))",0,0,1.239266e+10,4.957819e+10,0.249962
2,"POLYGON ((2 0, 1 0, 1 1, 2 1, 2 0))",0,1,1.239266e+10,4.957819e+10,0.249962
3,"POLYGON ((1 1, 1 2, 2 2, 2 1, 1 1))",0,2,1.239643e+10,4.957819e+10,0.250038


In [126]:
1.239266e+10/4.957819e+10

0.24996192882394455

In [99]:
    return gp.pd.DataFrame(itxns[[pname1, pname2, 'weight']])

,poly1,poly2,weight
0,1,3,1.000000
1,0,0,0.333333
2,0,1,0.333333
3,0,2,0.333333
